In [1]:
import os
import re
import sys
from openai import AzureOpenAI
import pandas as pd
import numpy as np
# import pymssql
from dotenv import load_dotenv, find_dotenv
import warnings 
import pyodbc
import mlflow
warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())
import ssl
import urllib3
import httpx

# Disable SSL certificate verification globally (for development only!)
ssl._create_default_https_context = ssl._create_unverified_context

# For requests and urllib3, suppress warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
os.environ.get("AZURE_OPENAI_ENDPOINT")

'https://20892-m7iyjxep-eastus2.cognitiveservices.azure.com/'

In [3]:
os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

'gpt-4.1'

In [4]:
os.environ.get("AZURE_OPENAI_VERSION")

'2025-01-01-preview'

In [5]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

# Chat Completions : Older models

In [12]:
import ssl
import urllib3
import httpx

# Disable SSL certificate verification globally (for development only!)
ssl._create_default_https_context = ssl._create_unverified_context

# For requests and urllib3, suppress warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from openai import AzureOpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_MODEL")

# Create httpx client with SSL verification disabled
http_client = httpx.Client(verify=False)

# Create client with custom http_client
client = AzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=API_ENDPOINT,
    api_version=API_VERSION,
    http_client=http_client
)

try:
    # Create completions with streaming enabled
    completion = client.chat.completions.create(
        model=AZURE_DEPLOYMENT,  # Use deployment name, not model name
        stream=True,
        messages=[
            {
                "role": "user",
                "content": "Tell me a long joke",
            },
        ],
    )

    print("API call successful! Response:")
    for chunk in completion:
        try:
            if hasattr(chunk.choices[0].delta, 'content') and chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end="")
        except (AttributeError, IndexError):
            pass
    print("\n")  # Add newline at the end

except Exception as e:
    print(f"Error making API call: {e}")
    print(f"API Endpoint: {API_ENDPOINT}")
    print(f"API Version: {API_VERSION}")
    print(f"Deployment: {AZURE_DEPLOYMENT}")

API call successful! Response:
Sure! Here’s a long joke for you:

A man is shipwrecked and finds himself stranded on a deserted island. Days turn into weeks, and finally one day he spots a bottle washed up on the shore. Excited, he opens it, and—out pops a genie!

The genie says, “Thank you for releasing me! For your kindness, I will grant you **three wishes**. But with each wish, whatever you ask for, your worst enemy will receive **double**.”

The man thinks a moment. “Okay,” he says, “For my first wish, I want a lavish villa to live in, right here on the island.”

The genie snaps his fingers. Instantly, a beautiful mansion appears next to him—fully furnished, with a swimming pool and everything. But then, on a distant island, his enemy’s villa appears, twice as large, twice as fancy.

“For my second wish,” says the man, “I want billions of dollars in gold.”

The genie snaps his fingers again. The man is surrounded by mountains of gold coins and bars. His bank account instantly fills

In [ ]:
import ssl
import urllib3
import httpx

# Disable SSL certificate verification globally (for development only!)
ssl._create_default_https_context = ssl._create_unverified_context

# For requests and urllib3, suppress warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from openai import AzureOpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_MODEL")

# Create httpx client with SSL verification disabled
http_client = httpx.Client(verify=False)

# Create client with custom http_client
client = AzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=API_ENDPOINT,
    api_version=API_VERSION,
    http_client=http_client
)

try:
    # Create completions with streaming enabled
    completion = client.chat.completions.create(
        model=AZURE_DEPLOYMENT,  # Use deployment name, not model name
        stream=False,
        messages=[
            {
                "role": "user",
                "content": "Tell me a long joke",
            },
        ],
    )
    print("API call successful! Response:")
    print(completion.choices[0].message.content)

except Exception as e:
    print(f"Error making API call: {e}")
    print(f"API Endpoint: {API_ENDPOINT}")
    print(f"API Version: {API_VERSION}")
    print(f"Deployment: {AZURE_DEPLOYMENT}")

Error making API call: Connection error.
API Endpoint: https://20892-m7iyjxep-eastus2.cognitiveservices.azure.com/
API Version: 2025-01-01-preview
Deployment: gpt-4.1


In [6]:

# Create client
client = AzureOpenAI(
  default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
  api_key=API_KEY,
  azure_endpoint=API_ENDPOINT,
  azure_deployment= AZURE_DEPLOYMENT,
  api_version=API_VERSION, 
)

# Create completions
completion = client.chat.completions.create(
  model=MODEL, 
  stream=True,
  messages=[
    {
      "role": "user",
      "content": "Tell me a long joke",
    },
  ],
)

for chunk in completion:
  try:
    print(chunk.choices[0].delta.content or "", end="")
  except Exception:
    pass


APIConnectionError: Connection error.

In [7]:

# Create client
client = AzureOpenAI(
  default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
  api_key=API_KEY,
  azure_endpoint=API_ENDPOINT,
  azure_deployment= AZURE_DEPLOYMENT,
  api_version=API_VERSION, 
)


In [8]:

# Create completions
completion = client.chat.completions.create(
  model=MODEL, 
  stream=True,
  messages=[
    {
      "role": "user",
      "content": "Tell me a long joke",
    },
  ],
)

for chunk in completion:
  try:
    print(chunk.choices[0].delta.content or "", end="")
  except Exception:
    pass


2025/06/16 09:33:42 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Absolutely! Here’s a long joke for you. Sit back and enjoy:

---

A man is driving down a country road when he sees a sign:  
TALKING DOG FOR SALE – INQUIRE WITHIN

Curious, he pulls in, walks up to the farmhouse, and knocks on the door. The owner greets him, and the man asks, “I saw your sign about a talking dog; is that for real?”

The farmer nods and points to the backyard. The man steps through the gate, and sure enough, there’s an old dog sitting calmly on the porch. He hesitates, then says, “Excuse me… are you the talking dog?”

The dog looks up and says, “Yep, that’s me.”

The man’s jaw drops. “What? You can really talk?!”

The dog nods. “Absolutely. I discovered it when I was young. It got me recruited by the CIA. I traveled the world, sat in on high-level meetings—no one ever suspected a dog was fluent in multiple languages. After that, I settled down here to relax. Now I just guard the porch, take naps, and give belly rubs to myself.”

The man, stunned, runs back to the farme

In [ ]:
def generate_response(prompt):
    completions = client.chat.completions.create(
        model=MODEL,
        stream=True,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    for chunk in completions:
        try:
            # print(chunk.choices[0].delta.content or "", end="")
            print(chunk.choices[0].delta.content or "", end="")
        except Exception:
            pass

In [ ]:
print(generate_response("What makes AI agents AI agents?"))

Great question! To answer **what makes AI agents 'AI agents'**, let's break it down:

### 1. **What is an Agent?**
In artificial intelligence (AI), **an agent is anything that perceives its environment and acts upon it**. It can be as simple as a thermostat (perceives temperature, turns heater on/off) or as complex as an autonomous car.

### 2. **What makes an Agent an AI Agent?**
An **AI agent** is specifically a *computational* agent that uses **artificial intelligence techniques** to operate. This means:

- **Perceives**: It gets input from the environment (via sensors or data).
- **Thinks/Reasons**: It processes that input intelligently (using logic, learning, planning, etc).
- **Acts**: It acts back on the environment (via effectors or commands).
- **Goal-Oriented**: It typically works towards a goal, optimizing its actions to succeed.

### 3. **Key Features That Make an Agent an AI Agent**
- **Autonomy**: Makes at least some decisions on its own, with limited human intervention.


# SQL Queries

In [ ]:
# to the RSVR
server_reservior = os.environ.get("rsvr")
rsvr_password = os.environ.get("rsvr_password")
rsvr_user =  os.environ.get("rsvr_user") #"SBICZA01\A238737"

# Connecting to the Resevoir
conn_reservior = pymssql.connect(server = server_reservior,
                                user = rsvr_user, 
                                password = rsvr_password)
cursor_rsvr = conn_reservior.cursor()

In [ ]:
# Query to get the data from the Reservoir
# df_data = pd.read_sql_query("""
#                             SELECT top 1000 * 
#                             FROM [RSVRRealtimeFinacleLS].[PLSFIN].[vwCuratedLSOFinacleTransactionsHistoryImpl] WITH (NOLOCK)
#                             """, conn_reservior)
# cursor_rsvr.close()
# conn_reservior.close()

# df_data.head()

In [ ]:
import re
import pandas as pd
import pymssql
import os
from tqdm import tqdm

def validate_sql_query(query):
    """
    Validates SQL query to ensure it's safe and only contains SELECT statements.
    Returns (is_valid, error_message).
    """
    import re
    
    # Normalize query - remove comments and extra whitespace
    normalized_query = re.sub(r'--.*', '', query)  # Remove line comments
    normalized_query = re.sub(r'/\*.*?\*/', '', normalized_query, flags=re.DOTALL)  # Remove block comments
    normalized_query = ' '.join(normalized_query.split()).upper()  # Normalize whitespace and convert to uppercase
    
    # List of dangerous SQL keywords that should be blocked
    dangerous_keywords = [
        'INSERT', 'UPDATE', 'DELETE', 'DROP', 'CREATE', 'ALTER', 'TRUNCATE',
        'MERGE', 'REPLACE', 'EXEC', 'EXECUTE', 'CALL', 'DECLARE', 'SET',
        'GRANT', 'REVOKE', 'COMMIT', 'ROLLBACK', 'SAVEPOINT', 'BEGIN',
        'TRANSACTION', 'LOCK', 'UNLOCK', 'BACKUP', 'RESTORE', 'SHUTDOWN',
        'USE', 'SHOW', 'DESCRIBE', 'EXPLAIN', 'LOAD', 'OUTFILE', 'INFILE',
        'IMPORT', 'EXPORT', 'BULK', 'OPENROWSET', 'OPENDATASOURCE'
    ]
    
    # Check for dangerous keywords
    for keyword in dangerous_keywords:
        if re.search(r'\b' + keyword + r'\b', normalized_query):
            return False, f"Dangerous SQL keyword detected: {keyword}"
    
    # Ensure query starts with SELECT
    if not normalized_query.strip().startswith('SELECT'):
        return False, "Only SELECT statements are allowed"
    
    # Check for suspicious patterns
    suspicious_patterns = [
        r';\s*(INSERT|UPDATE|DELETE|DROP|CREATE|ALTER)',  # Multiple statements
        r'UNION.*?(INSERT|UPDATE|DELETE|DROP|CREATE|ALTER)',  # Union injection
        r'OR\s+1\s*=\s*1',  # SQL injection pattern
        r'AND\s+1\s*=\s*1',  # SQL injection pattern
        r"'.*?'.*?;",  # Potential injection with semicolon
    ]
    
    for pattern in suspicious_patterns:
        if re.search(pattern, normalized_query, re.IGNORECASE):
            return False, f"Suspicious SQL pattern detected: {pattern}"
    
    return True, "Query is valid"

def execute_multiple_sql_code(sql_code, connection=None, connection_type="mssql"):
    """
    Accepts a string containing one or more SQL queries, each enclosed in triple backticks.
    Executes each query sequentially and stores the results in a list.
    
    Parameters:
    - sql_code: String containing SQL queries in ```sql code blocks
    - connection: Database connection object (pymssql, sqlite3, etc.)
    - connection_type: Type of database connection ("mssql", "sqlite", "mysql", "postgresql")
    
    Returns a list of results (SELECT returns DataFrame as JSON, validation errors return error messages).
    """
    import pandas as pd
    import json
    import re
    
    if connection is None:
        return [{"query": "", "result": "Error: No database connection provided", "status": "connection_error"}]
    
    # Improved regex to handle various backtick formats
    # This covers both ```sql and ``` sql (with spaces) and handles multiline blocks better
    queries = re.findall(r'```\s*sql\s*(.*?)```', sql_code, re.DOTALL)
    results = []
    
    if not queries:
        # Try more lenient pattern (without sql keyword)
        queries = re.findall(r'```(.*?)```', sql_code, re.DOTALL)
        
        if not queries:
            return [{"query": sql_code, 
                    "result": "No SQL queries found in the provided code. Please format queries in ```sql code blocks.",
                    "status": "format_error"}]
    
    for query_idx, query in enumerate(queries):
        query = query.strip()
        if not query:
            results.append({
                "query": "", 
                "result": f"Empty query found in code block #{query_idx+1}",
                "status": "validation_error"
            })
            continue
            
        # Validate the query first
        is_valid, validation_message = validate_sql_query(query)
        
        if not is_valid:
            results.append({
                "query": query, 
                "result": f"Query validation failed: {validation_message}",
                "status": "validation_error"
            })
            continue
        
        try:
            # Execute only SELECT statements (already validated)
            print(f"Running Query #{query_idx+1} ______________________")
            print(f"Executing query: {query[:100]}{'...' if len(query) > 100 else ''}")
            
            # Execute query
            df = pd.read_sql_query(query, connection)
            
            print(f"Query executed successfully - returned {len(df)} rows with {len(df.columns)} columns")
            
            # Handle empty result sets gracefully
            if len(df) == 0:
                results.append({
                    "query": query,
                    "result": "[]",  # Empty JSON array
                    "status": "success",
                    "row_count": 0,
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
                continue
            
            # Convert to JSON with error handling
            try:
                json_result = df.to_json(orient='records', date_format='iso')
                results.append({
                    "query": query, 
                    "result": json_result,
                    "status": "success",
                    "row_count": len(df),
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
            except Exception as json_err:
                # Fallback for JSON conversion errors
                results.append({
                    "query": query, 
                    "result": f"Error converting results to JSON: {str(json_err)}",
                    "status": "json_error",
                    "row_count": len(df),
                    "column_info": {col: str(dtype) for col, dtype in df.dtypes.items()}
                })
                
        except Exception as e:
            results.append({
                "query": query, 
                "result": f"Error executing SQL: {str(e)}",
                "status": "execution_error"
            })
    
    return results

def execute_sql_with_mssql_connection(sql_code):
    """
    Execute SQL queries using the existing MSSQL connection setup.
    """
    try:
        # Use your existing connection setup
        server_reservior = os.environ.get("rsvr")
        rsvr_password = os.environ.get("rsvr_password")
        rsvr_user = os.environ.get("rsvr_user")
        
        if not all([server_reservior, rsvr_password, rsvr_user]):
            return [{"query": "", 
                    "result": "Missing environment variables for database connection. Check rsvr, rsvr_password, rsvr_user", 
                    "status": "configuration_error"}]
        
        conn_reservior = pymssql.connect(
            server=server_reservior,
            user=rsvr_user, 
            password=rsvr_password
        )
        
        # Execute the queries
        results = execute_multiple_sql_code(sql_code, conn_reservior, "mssql")
        
        # Close connection
        conn_reservior.close()
        
        return results
        
    except Exception as e:
        return [{"query": "", "result": f"Connection error: {str(e)}", "status": "connection_error"}]

In [ ]:
def execute_multiple_sql_code(sql_code, connection=None, connection_type="mssql", use_raw_cursor=False):
    import pandas as pd
    import json
    import re

    if connection is None:
        return [{"query": "", "result": "Error: No database connection provided", "status": "connection_error"}]

    queries = re.findall(r'```\s*sql\s*(.*?)```', sql_code, re.DOTALL)
    results = []

    if not queries:
        queries = re.findall(r'```(.*?)```', sql_code, re.DOTALL)
        if not queries:
            return [{"query": sql_code, 
                     "result": "No SQL queries found in the provided code. Please format queries in ```sql code blocks.",
                     "status": "format_error"}]

    for query_idx, query in enumerate(queries):
        query = query.strip()
        if not query:
            results.append({
                "query": "", 
                "result": f"Empty query found in code block #{query_idx+1}",
                "status": "validation_error"
            })
            continue

        is_valid, validation_message = validate_sql_query(query)
        if not is_valid:
            results.append({
                "query": query, 
                "result": f"Query validation failed: {validation_message}",
                "status": "validation_error"
            })
            continue

        try:
            print(f"Running Query #{query_idx+1} ______________________")
            print(f"Executing query: {query[:100]}{'...' if len(query) > 100 else ''}")

            if use_raw_cursor:
                cursor = connection.cursor()
                cursor.execute(query)
                columns = [desc[0] for desc in cursor.description]
                rows = cursor.fetchall()
                df = pd.DataFrame(rows, columns=columns)
            else:
                df = pd.read_sql_query(query, connection)

            if len(df) == 0:
                results.append({
                    "query": query,
                    "result": "[]",
                    "status": "success",
                    "row_count": 0,
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
                continue

            try:
                json_result = df.to_json(orient='records', date_format='iso')
                results.append({
                    "query": query, 
                    "result": json_result,
                    "status": "success",
                    "row_count": len(df),
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
            except Exception as json_err:
                results.append({
                    "query": query, 
                    "result": f"Error converting results to JSON: {str(json_err)}",
                    "status": "json_error",
                    "row_count": len(df),
                    "column_info": {col: str(dtype) for col, dtype in df.dtypes.items()}
                })

        except Exception as e:
            results.append({
                "query": query, 
                "result": f"Error executing SQL: {str(e)}",
                "status": "execution_error"
            })

    return results


In [ ]:
# Example usage:
sql_code_example = """
```sql
SELECT top 100 * 
FROM [RSVRRealtimeFinacleLS].[PLSFIN].[vwCuratedLSOFinacleTransactionsHistoryImpl] WITH (NOLOCK);
```
"""


# LocalDB: pyodbc

In [2]:
server = os.environ.get("server_data_studio") 
database = os.environ.get("database")
password = os.environ.get("password")
username =  os.environ.get("username") #"SBICZA01\A238737"

In [3]:
server, database, username

('localhost\\SQLEXPRESS', 'master', 'A238737')

In [4]:
import pyodbc

# List available drivers
print("Available drivers:")
for driver in pyodbc.drivers():
    print(f"  - {driver}")

Available drivers:
  - SQL Server
  - ODBC Driver 17 for SQL Server
  - SQL Server Native Client RDA 11.0
  - Microsoft Access Driver (*.mdb, *.accdb)
  - Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
  - Microsoft Access Text Driver (*.txt, *.csv)
  - Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)


## Connect to satabase

In [5]:
# import connect_to_sql_server module
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

try:
    from db.sql_connector import connect_to_sql_server
    from db.sql_query_executor import execute_sql_with_pyodbc
    print("Successfully imported")
except ImportError as e:
    print(f"Error importing: {e}")
    print("Ensure the file/s is in the parent directory of your current script.")
    sys.exit(1)


Successfully imported


In [10]:
# Example usage:
# Windows authentication
conn, cursor = connect_to_sql_server(server, database, auth_type='windows')

# SQL Server authentication
# conn, cursor = connect_to_sql_server(server, database, auth_type='sql', username= username, password=password)

# Don't forget to close the connection when done
# if conn:
#     conn.close()

No .env file found. Will use provided parameters or defaults.
Available SQL Server drivers: ODBC Driver 17 for SQL Server, SQL Server, SQL Server Native Client RDA 11.0
Attempting connection: windows auth using ODBC Driver 17 for SQL Server
Server: localhost\SQLEXPRESS, Database: master
✓ Successfully connected to SQL Server
  Server: localhost\SQLEXPRESS
  Database: master
  Authentication: windows
  Driver: ODBC Driver 17 for SQL Server


In [11]:
try:
    query = """
    SELECT TOP 100 * 
    FROM [master].[dbo].[customer_information] WITH (NOLOCK);
    """
    df_pandas = pd.read_sql_query(query, conn)
    display(df_pandas.head())
except Exception as e:
    print(f"Error executing query: {e}")

,id,full_name,email,phone_number,address,account_number,account_type,balance,gender,age,...,loan_amount_applied_for,loan_status,loan_tenure,loan_interest_rate,loan_purpose,loan_application_date,last_login,product_holding,no_product_holding,entrenchment_score
0,10067724,Paula Gonzales,adam09@example.org,(854)608-8053,"31026 Elizabeth Squares Apt. 357, Evansland, S...",1144700244,Business,3522.19,Male,72,...,9113.06,Pending,24.0,10.34,Vehicle,2024-09-25 13:46:35,2024-12-18 13:25:39,"[""Certificates of Deposit (CDs)"", ""Brokerage A...",10,43.48
1,10236697,Crystal Mayo,leemike@example.net,516.613.2503x620,"048 Vazquez Plains, Peterbury, KY 76257",689676928,Business,36295.28,Male,31,...,0.00,None,NaN,NaN,None,NaT,2025-03-31 02:26:54,"[""Certificates of Deposit (CDs)"", ""Retirement ...",3,13.04
2,10285422,Lindsey Davis,zacharybaker@example.org,(325)803-7553x6772,"04781 Kelly Ville Suite 382, Andrewsbury, WY 9...",6365367477,Savings,28218.26,Female,55,...,0.00,None,NaN,NaN,None,NaT,2024-09-08 12:08:27,"[""Safe Deposit Boxes"", ""Credit Cards"", ""Saving...",7,30.43
3,10406511,Shelly Hernandez,cookdiane@example.com,264-669-3151,"325 Jessica Greens Suite 275, Port Aprilfurt, ...",5108727028,Business,47836.64,Female,58,...,4115.98,Approved,9.0,11.37,Home,2023-10-18 17:18:19,2025-02-19 01:39:36,"[""Overdraft Protection"", ""Foreign Currency Exc...",12,52.17
4,57305723,Alejandro Sims,reyesjared@example.com,299-339-8079x17778,"4658 Snyder Landing Suite 653, East Christophe...",6455579108,Business,28142.83,Female,60,...,0.00,None,NaN,NaN,None,NaT,2025-03-29 17:59:33,"[""Credit Cards"", ""Home Mortgages"", ""Auto Loans...",5,21.74


In [12]:
try:
    query = """
    SELECT TOP 100 * 
    FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
    """
    df_pandas = pd.read_sql_query(query, conn)
    display(df_pandas.head())
except Exception as e:
    print(f"Error executing query: {e}")

,transaction_id,customer_id,transaction_date,transaction_type,amount,currency,description,category,channel,status,reference_number,merchant_name,merchant_category,account_from,account_to,balance_after,location,created_at,updated_at
0,137676085,72592271,2025-06-08 11:29:15,Refund,8688.23,ZAR,Refund - Refund,Refund,Online Banking,Pending,REF6314Bk0120,None,None,6768494328,None,10943.45,"Gonzalezton, Bermuda",2025-06-09 20:44:55,2025-06-09 20:44:55
1,323151607,72696383,2025-05-18 10:48:40,Interest,85.04,ZAR,Interest - Investment,Investment,Branch,Pending,REF4902Vo9993,None,None,153279519,None,16167.80,"East Danielside, Hong Kong",2025-06-09 20:44:56,2025-06-09 20:44:56
2,425991815,38285644,2025-04-14 00:47:58,Interest,765.52,ZAR,Interest - Insurance,Insurance,ACH,Failed,REF4463MB2350,None,None,5103145505,None,3021.18,"Johnsonland, Turkmenistan",2025-06-09 20:44:56,2025-06-09 20:44:56
3,485118858,85824178,2023-08-30 00:09:12,Loan Payment,-7760.13,EUR,Loan Payment - Shopping,Shopping,ATM,Failed,REF1586BH1647,None,None,3476035664,None,44766.26,"South Veronicabury, Saudi Arabia",2025-06-09 20:44:56,2025-06-09 20:44:56
4,848137932,53536452,2023-09-06 01:26:15,Loan Payment,-4450.32,ZAR,Loan Payment - Gas,Gas,Branch,Cancelled,REF3560Qk4253,None,None,6284806913,None,14417.54,"Murphychester, Brazil",2025-06-09 20:44:56,2025-06-09 20:44:56


In [13]:
try:# Option 2: Using cursor directly
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    # Create DataFrame from cursor results
    df_cursor = pd.DataFrame.from_records(rows, columns=columns)
    # Display the DataFrame
    display(df_cursor.head())
except Exception as e:
    print(f"Error executing query with cursor: {e}")

,transaction_id,customer_id,transaction_date,transaction_type,amount,currency,description,category,channel,status,reference_number,merchant_name,merchant_category,account_from,account_to,balance_after,location,created_at,updated_at
0,137676085,72592271,2025-06-08 11:29:15,Refund,8688.23,ZAR,Refund - Refund,Refund,Online Banking,Pending,REF6314Bk0120,None,None,6768494328,None,10943.45,"Gonzalezton, Bermuda",2025-06-09 20:44:55,2025-06-09 20:44:55
1,323151607,72696383,2025-05-18 10:48:40,Interest,85.04,ZAR,Interest - Investment,Investment,Branch,Pending,REF4902Vo9993,None,None,153279519,None,16167.80,"East Danielside, Hong Kong",2025-06-09 20:44:56,2025-06-09 20:44:56
2,425991815,38285644,2025-04-14 00:47:58,Interest,765.52,ZAR,Interest - Insurance,Insurance,ACH,Failed,REF4463MB2350,None,None,5103145505,None,3021.18,"Johnsonland, Turkmenistan",2025-06-09 20:44:56,2025-06-09 20:44:56
3,485118858,85824178,2023-08-30 00:09:12,Loan Payment,-7760.13,EUR,Loan Payment - Shopping,Shopping,ATM,Failed,REF1586BH1647,None,None,3476035664,None,44766.26,"South Veronicabury, Saudi Arabia",2025-06-09 20:44:56,2025-06-09 20:44:56
4,848137932,53536452,2023-09-06 01:26:15,Loan Payment,-4450.32,ZAR,Loan Payment - Gas,Gas,Branch,Cancelled,REF3560Qk4253,None,None,6284806913,None,14417.54,"Murphychester, Brazil",2025-06-09 20:44:56,2025-06-09 20:44:56


## Multiple SQL Queries Execution

In [ ]:
# Example usage:
sql_code_example = """
```sql
SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
```
```sql
SELECT distinct transaction_id 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
```

"""

# Execute the example (uncomment to test)
results = execute_sql_with_pyodbc(sql_code_example, auth_type='windows')
for res in results:
    print(f"Status: {res['status']}")
    print(f"Query: {res['query'][:100]}...")
    if res['status'] == 'success':
        print(f"Rows returned: {res['row_count']}")
    print(f"Result: {res['result'][:200]}...")
    print("-" * 50)

No .env file found. Will use provided parameters or defaults.
Available SQL Server drivers: ODBC Driver 17 for SQL Server, SQL Server, SQL Server Native Client RDA 11.0
Attempting connection: windows auth using ODBC Driver 17 for SQL Server
Server: localhost\SQLEXPRESS, Database: master
✓ Successfully connected to SQL Server
  Server: localhost\SQLEXPRESS
  Database: master
  Authentication: windows
  Driver: ODBC Driver 17 for SQL Server
Running Query #1 ______________________
Executing query: -- 1. Quarterly highest and lowest transaction amount
WITH QuarterlySummary AS (
    SELECT 
        'Q' + CAST(DATEPART(QUARTER, transaction_date) AS VARCHAR) + ' ' + 
        CAST(DATEPART(YEAR, transaction_date) AS VARCHAR) AS quarter_year,
        SUM(amount) AS total_amount
    FROM [master].[dbo].[transaction_history] WITH (NOLOCK)
    WHERE status = 'Completed'
    GROUP BY DATEPART(YEAR, t...


Query executed successfully - returned 2 rows with 3 columns
Status: success
Query: -- 1. Quar

In [27]:
results

[{'query': "-- 1. Quarterly highest and lowest transaction amount\nWITH QuarterlySummary AS (\n    SELECT \n        'Q' + CAST(DATEPART(QUARTER, transaction_date) AS VARCHAR) + ' ' + \n        CAST(DATEPART(YEAR, transaction_date) AS VARCHAR) AS quarter_year,\n        SUM(amount) AS total_amount\n    FROM [master].[dbo].[transaction_history] WITH (NOLOCK)\n    WHERE status = 'Completed'\n    GROUP BY DATEPART(YEAR, transaction_date), DATEPART(QUARTER, transaction_date)\n),\nRankedQuarters AS (\n    SELECT \n        quarter_year,\n        total_amount,\n        RANK() OVER (ORDER BY total_amount DESC) AS highest_rank,\n        RANK() OVER (ORDER BY total_amount ASC) AS lowest_rank\n    FROM QuarterlySummary\n)\nSELECT \n    quarter_year,\n    total_amount,\n    CASE \n        WHEN highest_rank = 1 THEN 'Highest Amount'\n        WHEN lowest_rank = 1 THEN 'Lowest Amount'\n    END AS amount_category\nFROM RankedQuarters\nWHERE highest_rank = 1 OR lowest_rank = 1\nORDER BY total_amount DESC

In [13]:
def display_sql_results(results, max_query_preview=200, max_result_preview=100):
    """
    Display SQL query results in a formatted way.
    
    Parameters:
    - results: List of result dictionaries from execute_sql_with_pyodbc
    - max_query_preview: Maximum number of characters to show in query preview
    - max_result_preview: Maximum number of characters to show in result preview
    
    Returns:
    - None (prints results to console)
    """
    if not results:
        print("No results to display.")
        return
        
    for idx, res in enumerate(results, 1):
        # Create a separator with query number
        print(f"\n{'=' * 20} QUERY #{idx} {'=' * 20}")
        
        # Status with color indicators (using ANSI escape codes)
        status = res['status']
        status_color = {
            'success': '\033[92m',      # Green
            'validation_error': '\033[93m',  # Yellow
            'execution_error': '\033[91m',   # Red
            'connection_error': '\033[91m',  # Red
            'json_error': '\033[93m',        # Yellow
            'format_error': '\033[93m',      # Yellow
            'configuration_error': '\033[91m' # Red
        }.get(status, '')
        end_color = '\033[0m'  # Reset color
        
        print(f"Status: {status_color}{status}{end_color}")
        
        # Query preview with ellipsis if truncated
        query = res.get('query', '')
        query_preview = query[:max_query_preview]
        if len(query) > max_query_preview:
            query_preview += "..."
        print(f"Query: {query_preview}")
        
        # For successful queries, show row and column info
        if status == 'success':
            print(f"Rows: {res.get('row_count', 'N/A')} | Columns: {res.get('column_count', 'N/A')}")
            if 'columns' in res:
                print(f"Column names: {', '.join(res['columns'])}")
        
        # Result preview with ellipsis if truncated
        result = res.get('result', '')
        result_preview = result[:max_result_preview]
        if len(result) > max_result_preview:
            result_preview += "..."
        print(f"Result: {result_preview}")
        
        # End of result separator
        print("-" * 50)

In [14]:
display_sql_results(results)


==================== QUERY #1 ====================
Status: success
Query: SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
Rows: 1000 | Columns: 19
Column names: transaction_id, customer_id, transaction_date, transaction_type, amount, currency, description, category, channel, status, reference_number, merchant_name, merchant_category, account_from, account_to, balance_after, location, created_at, updated_at
Result: [{"transaction_id":137676085,"customer_id":72592271,"transaction_date":"2025-06-08T11:29:15.000","tr...
--------------------------------------------------

==================== QUERY #2 ====================
Status: success
Query: SELECT distinct transaction_id 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
Rows: 7000 | Columns: 1
Column names: transaction_id
Result: [{"transaction_id":137676085},{"transaction_id":323151607},{"transaction_id":425991815},{"transactio...
--------------------------------------------------


# Agents

In [6]:
from pathlib import Path

import os
import re
import sys
from openai import AzureOpenAI
import pandas as pd
import numpy as np
import json
from dotenv import load_dotenv, find_dotenv
import warnings 
warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

# Add the src directory to the Python path
# src_dir = str(Path(__file__).parent.parent)
# sys.path.append(src_dir)
# Add the parent directory to sys.path to allow imports from rag-compliance

True

In [7]:

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Now import the modules from rag-compliance
try:
    # Import prompts
    from prompts.prompt_agent_router import prompt_agent_router
    from prompts.prompt_agent_sql_analysis import prompt_agent_sql_analysis
    from agents.agents import routing_agent,  agent_sql_analysis, agent_final_response
    

    from tools.tools import tools_definitions
    
    # Import agents (if any)
    # from rag_compliance.agents.your_agent_module import your_agent_function
    
    print("Successfully imported modules from rag-compliance directory")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Make sure the directory structure is correct and modules exist")
    
    # Debugging information
    print("\nDebugging information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Parent directory added to path: {parent_dir}")
    print(f"Full sys.path: {sys.path}")
    
    # List files in the expected directories to help with debugging
    rag_compliance_dir = os.path.join(parent_dir, 'rag_compliance')
    if os.path.exists(rag_compliance_dir):
        print(f"\nContents of {rag_compliance_dir}:")
        print(os.listdir(rag_compliance_dir))
        
        prompts_dir = os.path.join(rag_compliance_dir, 'prompts')
        if os.path.exists(prompts_dir):
            print(f"\nContents of {prompts_dir}:")
            print(os.listdir(prompts_dir))

Successfully imported modules from rag-compliance directory


## Tools

In [15]:
tools_definitions()

[{'type': 'function',
  'function': {'name': 'agent_sql_analysis',
   'description': "multi-intent sql analysis. Generates executable Microsoft SQL Server (MSSQL) queries based on user requests for database operations. The 'user_requests' parameter can be any data type (string, list, dictionary, tuple, JSON object, etc.), including natural language, structured instructions, or outputs from other agents. The function can handle complex queries involving multiple tables, aggregations, and conditions.The function should interpret the intent or intents and generate an appropriate SQL query.",
   'parameters': {'type': 'object',
    'properties': {'user_requests': {'type': 'string',
      'description': "The user's request/s, which can be any data type (string, list, dictionary, tuple, JSON object, etc.), including natural language, structured input, or the output from the router agent about what database operation to perform.user request can be multiple intents or single intents.Multi-inte

In [16]:
# def agent_sql_analysis(user_request=None, identifier=None):
#     pass

def agent_product_offerings(user_request=None, identifier=None):
    pass

def agent_file_validation(file_path=None, validation_type="standard"):
    pass

def agent_compare_files(baseline_data=None, comparison_data=None, comparison_type="standard"):
    pass
def agent_others(baseline_data=None, comparison_data=None, comparison_type="standard"):
    pass

## Router Agent

In [8]:
# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

### For Windows

In [9]:
# Create httpx client with SSL verification disabled
http_client = httpx.Client(verify=False)

# Create client with custom http_client
client = AzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=API_ENDPOINT,
    api_version=API_VERSION,
    http_client=http_client
)

In [ ]:
# Create client
# client = AzureOpenAI(
#   default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
#   api_key=API_KEY,
#   azure_endpoint=API_ENDPOINT,
#   azure_deployment= AZURE_DEPLOYMENT,
#   api_version=API_VERSION, 
# )


In [19]:

def generate_response(user_query):
    completions = client.chat.completions.create(
        model=MODEL,
        stream=True,
        
        messages=[
            {"role": "user", "content": user_query}
        ],
        # tools=tools_definitions(),
        # tool_choice="auto"
    )

    for chunk in completions:
        try:
            # print(chunk.choices[0].delta.content or "", end="")
            print(chunk.choices[0].delta.content or "", end="")
        except Exception:
            pass

print(generate_response("What makes AI agents AI agents?"))

Great question! Let’s break it down:

**An “AI agent” is, simply, any system that:**
1. **Operates autonomously** (acts on its own, without needing constant human guidance),
2. **Perceives its environment** (gathers information, e.g., via sensors or data input),
3. **Makes decisions** (uses reasoning or learned patterns to choose what to do),
4. **Acts** (takes actions to achieve goals),
5. **Often adapts over time** (learns from experience to improve).

### Core Characteristics

#### 1. **Autonomy**
   - An AI agent can operate by itself to some extent, making decisions and acting on them.

#### 2. **Perception**
   - The agent senses its environment (literal sensors, user inputs, web data, etc.).

#### 3. **Goal-directed behavior**
   - It acts to reach a goal (e.g., answering questions, playing a game, navigating a room).

#### 4. **Decision-making / Reasoning**
   - Uses algorithms (logic, machine learning, etc.) to decide what action to take next.

#### 5. **(Optional) Learning**


In [21]:
def agent_router(user_query):
    completions = client.chat.completions.create(
        model=MODEL,
        # stream=True,
        messages=[
            {"role": "system", "content": prompt_agent_router()},
            {"role": "user", "content": user_query}
        ],
        tools=tools_definitions(),
        tool_choice="auto"
    )
    return completions


In [22]:
# No tool call
response = agent_router("what is an ai agent?")

In [24]:
response = routing_agent("what is an ai agent?", chat_history=[])

In [25]:
response

ChatCompletion(id='chatcmpl-Bj1DAsf7FAi8fx7k3S8BJj4jFF6SX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='An AI agent is a system or software program that can perceive its environment, make decisions, and take actions to achieve specific goals using artificial intelligence techniques. It can process information, learn from data, and act autonomously or semi-autonomously to complete tasks. AI agents are used in various applications, such as virtual assistants, recommendation systems, autonomous vehicles, and customer support bots. They often combine techniques like machine learning, natural language processing, and rule-based reasoning to solve problems and interact with users or other systems.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'p

In [26]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='An AI agent is a system or software program that can perceive its environment, make decisions, and take actions to achieve specific goals using artificial intelligence techniques. It can process information, learn from data, and act autonomously or semi-autonomously to complete tasks. AI agents are used in various applications, such as virtual assistants, recommendation systems, autonomous vehicles, and customer support bots. They often combine techniques like machine learning, natural language processing, and rule-based reasoning to solve problems and interact with users or other systems.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'sel

In [27]:
print(response.choices[0].message.content)

An AI agent is a system or software program that can perceive its environment, make decisions, and take actions to achieve specific goals using artificial intelligence techniques. It can process information, learn from data, and act autonomously or semi-autonomously to complete tasks. AI agents are used in various applications, such as virtual assistants, recommendation systems, autonomous vehicles, and customer support bots. They often combine techniques like machine learning, natural language processing, and rule-based reasoning to solve problems and interact with users or other systems.


In [26]:
# tool call
response = agent_router("show me top 5 clients by account balance?")

In [12]:
# tool call
response = routing_agent("show me top 5 clients by account balance?", chat_history=[])

2025/06/16 12:28:26 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


In [13]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_E4SHRjmAr8gM10sW5rgMR9Lg', function=Function(arguments='{"user_request":"Retrieve the top 5 clients with the highest account balances from the customer_information table. Provide their full names and balance amounts, sorted by balance descending."}', name='agent_sql_analysis'), type='function')]), content_filter_results={})

In [14]:
output = response.choices[0].message
output

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_E4SHRjmAr8gM10sW5rgMR9Lg', function=Function(arguments='{"user_request":"Retrieve the top 5 clients with the highest account balances from the customer_information table. Provide their full names and balance amounts, sorted by balance descending."}', name='agent_sql_analysis'), type='function')])

In [15]:
print(output.content)

None


In [16]:
output.tool_calls[0]

ChatCompletionMessageToolCall(id='call_E4SHRjmAr8gM10sW5rgMR9Lg', function=Function(arguments='{"user_request":"Retrieve the top 5 clients with the highest account balances from the customer_information table. Provide their full names and balance amounts, sorted by balance descending."}', name='agent_sql_analysis'), type='function')

In [17]:
output.tool_calls[0].function.name

'agent_sql_analysis'

In [18]:
output.tool_calls[0].function.arguments

'{"user_request":"Retrieve the top 5 clients with the highest account balances from the customer_information table. Provide their full names and balance amounts, sorted by balance descending."}'

## agent_sql_analysis

In [20]:
def agent_sql_analysis(user_input):
    """
    Processes SQL query-related requests.
    
    Args:
        user_input: String or JSON-serializable input from the user
        
    Returns:
        str: The generated response text
    """
    # Accepts any data type: string, dict, list, etc.
    # If not string, convert to JSON string for the LLM
    if not isinstance(user_input, str):
        user_input_serialized = json.dumps(user_input)
    else:
        user_input_serialized = user_input

    completions = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt_agent_sql_analysis()},
            {"role": "user", "content": user_input_serialized}
        ],
    )
    return completions.choices[0].message.content



In [27]:
user_query = output.tool_calls[0].function.arguments
sql_output = agent_sql_analysis(user_query)

In [28]:
user_query

'{"user_request":"Retrieve the top 5 clients with the highest account balances from the customer_information table. Provide their full names and balance amounts, sorted by balance descending."}'

In [29]:
print(sql_output)

```sql
SELECT TOP 5 full_name, balance
FROM [dbo].[customer_information] WITH (NOLOCK)
ORDER BY balance DESC;
```


In [32]:
sql_results = execute_sql_with_pyodbc(sql_output)
display_sql_results(sql_results, max_query_preview=200, max_result_preview=100)

Loaded environment variables from /Users/mzwandilemhlongo/Desktop/Data Science/PersonalProjects/rag-system/rag-text2sql/.env
Available SQL Server drivers: ODBC Driver 18 for SQL Server, ODBC Driver 17 for SQL Server
Attempting connection: sql auth using ODBC Driver 18 for SQL Server
Server: localhost,1433, Database: master
✓ Successfully connected to SQL Server
  Server: localhost,1433
  Database: master
  Authentication: sql
  Driver: ODBC Driver 18 for SQL Server
Running Query #1 ______________________
Executing query: SELECT TOP 5 full_name, balance
FROM [dbo].[customer_information] WITH (NOLOCK)
ORDER BY balance DES...


Query executed successfully - returned 5 rows with 2 columns

==================== QUERY #1 ====================
Status: success
Query: SELECT TOP 5 full_name, balance
FROM [dbo].[customer_information] WITH (NOLOCK)
ORDER BY balance DESC;
Rows: 5 | Columns: 2
Column names: full_name, balance
Result: [{"full_name":"Madeline Blanchard","balance":49695.65},{"full_name

# Chat Loops

In [20]:
chat_history =[]

In [11]:
import json

# def routing_agent(user_request, chat_history):
#     """Routes the user request to the appropriate function/tool, using chat history for context."""
#     prompt = prompt_agent_router()

#     messages = [{"role": "system", "content": prompt}]
#     messages.extend(chat_history)

#     # Ensure latest user message is not duplicated
#     if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_request:
#         messages.append({"role": "user", "content": user_request})

#     response = client.chat.completions.create(
#         model=MODEL,
#         messages=messages,
#         tools=tools_definitions(),
#         tool_choice="auto"
#     )
#     return response

def ai_chatbot(user_input, chat_history):
    # Avoid duplicate user input
    if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_input:
        chat_history.append({"role": "user", "content": user_input})

    response = routing_agent(user_input, chat_history)

    message = response.choices[0].message

    if hasattr(message, 'tool_calls') and message.tool_calls:
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        print(f"Function Name: {function_name}")
        print(f"Function Arguments: {function_args}")
        print("\n")

        sql_results, product_offerings, validation_results = [], [], None

        if function_name == "agent_sql_analysis":
            sql_code = agent_sql_analysis(function_args["user_request"])
            print(f"SQL Code: {sql_code}")
            sql_results = execute_sql_with_pyodbc(sql_code)
            print(f"SQL Results: {sql_results}")

        elif function_name == "agent_product_offerings":
            product_offerings = [agent_product_offerings(function_args["user_request"])]

        elif function_name == "agent_file_validation":
            validation_results = agent_file_validation(function_args["file_path"])

        elif function_name == "agent_compare_files":
            validation_results = agent_compare_files(
                function_args["baseline_data"],
                function_args["comparison_data"],
                function_args.get("comparison_type", "standard")
            )

        else:
            assistant_msg = "I don't know how to handle that request."
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                chat_history.append({"role": "assistant", "content": assistant_msg})
            return chat_history

        # Process SQL results
        processed_sql_data = []
        for res in sql_results:
            if isinstance(res, dict) and res.get("status") == "success":
                try:
                    json_data = json.loads(res["result"])
                    processed_sql_data.append({
                        "query_info": f"Query returned {res['row_count']} rows with columns: {', '.join(res['columns'])}",
                        "data": json_data
                    })
                except json.JSONDecodeError:
                    processed_sql_data.append({
                        "query_info": f"Query returned {res['row_count']} rows",
                        "data": res["result"]
                    })
            else:
                processed_sql_data.append({
                    "query_info": f"Query failed with status: {res.get('status', 'unknown')}",
                    "data": res.get("result", "No data available")
                })

        # Build polish prompt
        if validation_results:
            polish_prompt = (
                "Please provide a clear, friendly response about the validation results: "
                f"{validation_results}. Format the response to be easy to understand."
            )
        elif sql_results and product_offerings:
            polish_prompt = (
                "The user asked for both data and product information. Provide a comprehensive response that:\n"
                f"1. Summarizes the data results: {processed_sql_data}\n"
                f"2. Describes the product offerings: {product_offerings}"
            )
        elif sql_results:
            polish_prompt = (
                "Summarize the following database query results in a clear, friendly way:\n"
                f"{processed_sql_data}"
            )
        elif product_offerings:
            polish_prompt = (
                "Present the following product information in a helpful way:\n"
                f"{product_offerings}"
            )
        else:
            assistant_msg = "I couldn't process your request. Please try again."
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                chat_history.append({"role": "assistant", "content": assistant_msg})
            return chat_history
        # Use polish_messages correctly
        polish_messages = [
            {"role": "system", "content": "You are a helpful banking assistant that provides clear, friendly responses to users."},
            {"role": "user", "content": polish_prompt}
        ]

        # polish_response = client.chat.completions.create(
        #     model=MODEL,
        #     messages=polish_messages
        # )

        polish_response = routing_agent(polish_prompt, chat_history)

        final_message = polish_response.choices[0].message.content
        if final_message and final_message.strip():
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != final_message:
                chat_history.append({"role": "assistant", "content": final_message})

    else:
        content = message.content or "I don't have a response for that."
        if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != content:
            chat_history.append({"role": "assistant", "content": content})

    return chat_history


In [10]:
from IPython.display import clear_output, display, Markdown
import sys

chat_history = []

def inline_input(prompt=""):
    """Custom input function that displays inline in Jupyter notebook"""
    print(prompt, end='')
    sys.stdout.flush()
    return input()

def display_chat(chat_history):
    """Nicely formats and displays the full chat history"""
    clear_output(wait=True)
    for msg in chat_history:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            display(Markdown(f"**You:** {content}"))
        elif role == "assistant":
            display(Markdown(f"**Assistant:** {content}"))

# Main loop
while True:
    display_chat(chat_history)
    user_input = inline_input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break

    # Process input and update chat history
    chat_history = ai_chatbot(user_input, chat_history)


You: Exiting chat.


In [22]:
chat_history

[{'role': 'user',
  'content': 'which quarter had the highest and lowest transaction amount?'},
 {'role': 'user', 'content': 'which client has the lowest balance?'}]

# MLFlow Integration

In [10]:
import mlflow
mlflow.openai.autolog()

In [11]:
# utils/mlflow_logger.py
import mlflow
import json

def start_chat_run(user_input):
    mlflow.set_experiment("Chatbot_Tracing")
    mlflow.start_run(nested=True)
    mlflow.log_param("user_input", user_input)

def log_router_response(message):
    mlflow.log_text(str(message), "router_response.json")

def log_sql_code(sql_code):
    mlflow.log_text(sql_code, "generated_sql.json")

def log_sql_results(results):
    mlflow.log_text(json.dumps(results, indent=2), "sql_results.json")

def log_product_info(info):
    mlflow.log_text(json.dumps(info, indent=2), "product_info.json")

def log_validation_result(result, filename="validation_result.txt"):
    mlflow.log_text(str(result), filename)

def log_final_response(response):
    mlflow.log_text(response, "final_response.txt")

def end_chat_run():
    mlflow.end_run()


In [12]:
from utils.mlflow_logger import (
    start_chat_run, log_router_response, log_sql_code, log_sql_results,
    log_product_info, log_validation_result, log_final_response, end_chat_run
)


In [13]:
def ai_chatbot(user_input, chat_history):
    if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_input:
        chat_history.append({"role": "user", "content": user_input})

    start_chat_run(user_input)

    try:
        response = routing_agent(user_input, chat_history)
        message = response.choices[0].message
        log_router_response(message)

        if hasattr(message, 'tool_calls') and message.tool_calls:
            tool_call = message.tool_calls[0]
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            # print(f"Function Name: {function_name}")
            print(f"\nFunction Arguments:\n {function_args}")

            sql_results, product_offerings, validation_results = [], [], None

            if function_name == "agent_sql_analysis":
                sql_code = agent_sql_analysis(function_args["user_requests"])
                log_sql_code(sql_code)
                # print(f"\n________________Generated SQL Code:\n {sql_code}") 
                sql_results = execute_sql_with_pyodbc(sql_code, auth_type='windows')
                # sql_results = execute_sql_with_pyodbc(sql_code)
                # print(f"\n\n__________SQL Results:________________\n {sql_results}")
                log_sql_results(sql_results)

            elif function_name == "agent_product_offerings":
                product_info = agent_product_offerings(function_args["user_request"])
                product_offerings = [product_info]
                log_product_info(product_info)

            elif function_name == "agent_file_validation":
                validation_results = agent_file_validation(function_args["file_path"])
                log_validation_result(validation_results, "file_validation.txt")

            elif function_name == "agent_compare_files":
                validation_results = agent_compare_files(
                    function_args["baseline_data"],
                    function_args["comparison_data"],
                    function_args.get("comparison_type", "standard")
                )
                log_validation_result(validation_results, "file_comparison.txt")

            else:
                assistant_msg = "I don't know how to handle that request."
                if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                    chat_history.append({"role": "assistant", "content": assistant_msg})
                return chat_history

            # Process SQL results
            processed_sql_data = []
            for res in sql_results:
                if isinstance(res, dict) and res.get("status") == "success":
                    try:
                        json_data = json.loads(res["result"])
                        processed_sql_data.append({
                            "query_info": f"Query returned {res['row_count']} rows with columns: {', '.join(res['columns'])}",
                            "data": json_data
                        })
                    except json.JSONDecodeError:
                        processed_sql_data.append({
                            "query_info": f"Query returned {res['row_count']} rows",
                            "data": res["result"]
                        })
                else:
                    processed_sql_data.append({
                        "query_info": f"Query failed with status: {res.get('status', 'unknown')}",
                        "data": res.get("result", "No data available")
                    })

            # Build polish prompt
            if sql_results:
                polish_prompt = (
                    f"""Summarize the following database query results in a clear, friendly way based on the user requests:
                    ** User Requests:** {function_args.get("user_requests")} \n
                    ** Results: {processed_sql_data}"""
                                                               
                )
                
            elif validation_results:
                polish_prompt = (
                    "Please provide a clear, friendly response about the validation results: "
                    f"{validation_results}. Format the response to be easy to understand."
                )
            elif product_offerings:
                polish_prompt = (
                    "Present the following product information in a helpful way:\n"
                    f"{product_offerings}"
                )
            else:
                assistant_msg = "I couldn't process your request. Please try again."
                if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                    chat_history.append({"role": "assistant", "content": assistant_msg})
                return chat_history

            # print(f"\nPolish Prompt:\n {polish_prompt}")
            
            # Use routing_agent but with a fresh chat history containing only the polish prompt
            # This prevents context confusion from the previous messages
            polish_chat_history = [
                {"role": "user", "content": polish_prompt}
            ]
            
            polish_response = agent_final_response(polish_prompt, polish_chat_history)
            # polish_response = routing_agent(polish_prompt, polish_chat_history)
            

            
            if polish_response and polish_response.choices and polish_response.choices[0].message:
                final_message = polish_response.choices[0].message.content
                # print(f"\nPolish Response: {final_message}")
                
                log_final_response(final_message)

                if final_message and final_message.strip():
                    if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != final_message:
                        chat_history.append({"role": "assistant", "content": final_message})
            else:
                # Fallback if polish_response is not properly formed
                fallback_message = "I've analyzed your request but encountered an issue formatting the results. Here's what I found: "
                if sql_results:
                    fallback_message += f"Query results show data for {len(sql_results)} SQL queries."
                chat_history.append({"role": "assistant", "content": fallback_message})
                log_final_response(fallback_message)

        else:
            content = message.content or "I don't have a response for that."
            log_final_response(content)
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != content:
                chat_history.append({"role": "assistant", "content": content})

    finally:
        end_chat_run()

    return chat_history

In [21]:
from IPython.display import clear_output, display, Markdown
import sys

chat_history = []

def inline_input(prompt=""):
    """Custom input function that displays inline in Jupyter notebook"""
    print(prompt, end='')
    sys.stdout.flush()
    return input()

def display_chat(chat_history):
    """Nicely formats and displays the full chat history"""
    clear_output(wait=True)
    for msg in chat_history:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            display(Markdown(f"**You:** {content}"))
        elif role == "assistant":
            display(Markdown(f"**Assistant:** {content}"))

# Main loop
while True:
    display_chat(chat_history)
    user_input = inline_input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break

    # Process input and update chat history
    chat_history = ai_chatbot(user_input, chat_history)


**You:** which quarter has the highest amount and the lowest amount? Which client has the lowest and highest balance? Tell me, whats an ai agents?

**Assistant:** Here’s a summary of your requested information:

1. **Quarters with Highest and Lowest Amounts:**
   - **Highest Amount:** Q4 2024, with a total amount of $1,608,701.77 from 244 transactions.
   - **Lowest Amount:** Q2 2023, with a total amount of $91,590.26 from 38 transactions.

2. **Clients with the Highest and Lowest Balances:**
   - **Highest Balance:** Charles Cline, with a balance of $49,859.96.
   - **Lowest Balance:** Christy Adams, with a balance of $158.62.

If you need more details or information on other quarters or clients, feel free to ask!

You: Exiting chat.


In [ ]:
which quarter has the highest amount and the lowest amount? Which client has the lowest and highest balance? Tell me, whats an ai agents?

In [34]:
chat_history

[{'role': 'user',
  'content': 'which quarter has the highest amount and the lowest amount? Which client has the lowest and highest balance? Tell me, whats an ai agents?'}]

In [14]:
chat_history = []
user_input = "Tell me, whats an ai agents? "
ai_chatbot(user_input, chat_history)

[{'role': 'user', 'content': 'Tell me, whats an ai agents? '},
 {'role': 'assistant',
  'content': 'An AI agent is a software program or system that uses artificial intelligence to perceive its environment, process information, make decisions, and take actions to achieve specific goals. In simple terms, an AI agent is like a digital assistant that can understand inputs (like language, data, or images), reason about them, and respond appropriately—often learning and improving over time.\n\nKey characteristics of an AI agent:\n- **Autonomy**: Acts independently based on goals or instructions.\n- **Perception**: Senses or gathers information about its environment (e.g., through sensors, data, or user input).\n- **Reasoning**: Processes and interprets this information, often using AI techniques like machine learning or rules.\n- **Action**: Takes actions or provides responses based on its analysis.\n- **Learning**: Some AI agents improve their performance through experience.\n\nExamples of

In [16]:
chat_history = []
session_id = "user_001"

In [17]:
user_input = "which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?"
ai_chatbot(user_input, chat_history)


Function Arguments:
 {'user_requests': '1. Which quarter has the highest amount and the lowest amount? 2. Which customer has the lowest and highest account balance? 3. Which channel has the highest transaction volume?'}
No .env file found. Will use provided parameters or defaults.
Available SQL Server drivers: ODBC Driver 17 for SQL Server, SQL Server, SQL Server Native Client RDA 11.0
Attempting connection: windows auth using ODBC Driver 17 for SQL Server
Server: localhost\SQLEXPRESS, Database: master
✓ Successfully connected to SQL Server
  Server: localhost\SQLEXPRESS
  Database: master
  Authentication: windows
  Driver: ODBC Driver 17 for SQL Server
Running Query #1 ______________________
Executing query: -- 1. Quarter with the highest and lowest total transaction amount
WITH QuarterlySummary AS (
    SELECT 
        DATEPART(YEAR, transaction_date) AS txn_year,
        DATEPART(QUARTER, transaction_date) AS txn_quarter,
        'Q' + CAST(DATEPART(QUARTER, transaction_date) AS V

[{'role': 'user',
  'content': 'which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?'},
 {'role': 'assistant',
  'content': "Here's a summary of your requested database insights:\n\n1. **Quarter with the Highest and Lowest Amounts:**\n   - **Highest Amount:** Q4 2024, with a total of 1,608,701.77.\n   - **Lowest Amount:** Q2 2023, with a total of 91,590.26.\n\n2. **Customers with the Highest and Lowest Account Balances:**\n   - **Highest Balance:** Charles Cline (Account: 5423480814), with a balance of 49,859.96.\n   - **Lowest Balance:** Christy Adams (Account: 2310140863), with a balance of 158.62.\n\n3. **Channel with the Highest Transaction Volume:**\n   - **Check** channel, with 205 transactions.\n\nLet me know if you need more details or further analysis!"}]

In [26]:

user_input = "Tell me, whats an ai agents? which quarter has the highest amount and the lowest amount? Which client has the lowest and highest balance? "
ai_chatbot(user_input, chat_history)


Function Arguments:
 {'user_requests': '1. Which quarter has the highest amount and the lowest amount? 2. Which client has the lowest and highest balance?'}
No .env file found. Will use provided parameters or defaults.
Available SQL Server drivers: ODBC Driver 17 for SQL Server, SQL Server, SQL Server Native Client RDA 11.0
Attempting connection: windows auth using ODBC Driver 17 for SQL Server
Server: localhost\SQLEXPRESS, Database: master
✓ Successfully connected to SQL Server
  Server: localhost\SQLEXPRESS
  Database: master
  Authentication: windows
  Driver: ODBC Driver 17 for SQL Server
Running Query #1 ______________________
Executing query: -- 1. Quarter with the highest and lowest total amount
WITH QuarterlySummary AS (
    SELECT 
        'Q' + CAST(DATEPART(QUARTER, transaction_date) AS VARCHAR) + ' ' + 
        CAST(DATEPART(YEAR, transaction_date) AS VARCHAR) AS quarter_year,
        SUM(amount) AS total_amount,
        COUNT(*) AS transaction_count
    FROM [master].[dbo

[{'role': 'user',
  'content': 'Tell me, whats an ai agents? which quarter has the highest amount and the lowest amount? Which client has the lowest and highest balance? '},
 {'role': 'assistant',
  'content': 'Here’s a summary based on your requests:\n\n1. **Quarter with the Highest and Lowest Amounts:**\n   - The quarter with the **highest total amount** is **Q4 2024** with **$1,608,701.77** across 244 transactions.\n   - The quarter with the **lowest total amount** is **Q2 2023** with **$91,590.26** across 38 transactions.\n\n2. **Client with the Highest and Lowest Balances:**\n   - The client with the **highest balance** is **Charles Cline**, with a balance of **$49,859.96**.\n   - The client with the **lowest balance** is **Christy Adams**, with a balance of **$158.62**.\n\nLet me know if you need more details or further breakdowns!'}]

In [19]:
user_input = "what was my previous question? "
ai_chatbot(user_input, chat_history)

[{'role': 'user',
  'content': 'which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?'},
 {'role': 'assistant',
  'content': "Here's a summary of your requested database insights:\n\n1. **Quarter with the Highest and Lowest Amounts:**\n   - **Highest Amount:** Q4 2024, with a total of 1,608,701.77.\n   - **Lowest Amount:** Q2 2023, with a total of 91,590.26.\n\n2. **Customers with the Highest and Lowest Account Balances:**\n   - **Highest Balance:** Charles Cline (Account: 5423480814), with a balance of 49,859.96.\n   - **Lowest Balance:** Christy Adams (Account: 2310140863), with a balance of 158.62.\n\n3. **Channel with the Highest Transaction Volume:**\n   - **Check** channel, with 205 transactions.\n\nLet me know if you need more details or further analysis!"},
 {'role': 'user', 'content': 'what was my previous question? '},
 {'role': 'assistant',
  'content': 'Your pr

In [20]:
user_input = "and what was your answer to that question? "
ai_chatbot(user_input, chat_history)

[{'role': 'user',
  'content': 'which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?'},
 {'role': 'assistant',
  'content': "Here's a summary of your requested database insights:\n\n1. **Quarter with the Highest and Lowest Amounts:**\n   - **Highest Amount:** Q4 2024, with a total of 1,608,701.77.\n   - **Lowest Amount:** Q2 2023, with a total of 91,590.26.\n\n2. **Customers with the Highest and Lowest Account Balances:**\n   - **Highest Balance:** Charles Cline (Account: 5423480814), with a balance of 49,859.96.\n   - **Lowest Balance:** Christy Adams (Account: 2310140863), with a balance of 158.62.\n\n3. **Channel with the Highest Transaction Volume:**\n   - **Check** channel, with 205 transactions.\n\nLet me know if you need more details or further analysis!"},
 {'role': 'user', 'content': 'what was my previous question? '},
 {'role': 'assistant',
  'content': 'Your pr

In [18]:
chat_history

[{'role': 'user',
  'content': 'which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?'},
 {'role': 'assistant',
  'content': "Here's a summary of your requested database insights:\n\n1. **Quarter with the Highest and Lowest Amounts:**\n   - **Highest Amount:** Q4 2024, with a total of 1,608,701.77.\n   - **Lowest Amount:** Q2 2023, with a total of 91,590.26.\n\n2. **Customers with the Highest and Lowest Account Balances:**\n   - **Highest Balance:** Charles Cline (Account: 5423480814), with a balance of 49,859.96.\n   - **Lowest Balance:** Christy Adams (Account: 2310140863), with a balance of 158.62.\n\n3. **Channel with the Highest Transaction Volume:**\n   - **Check** channel, with 205 transactions.\n\nLet me know if you need more details or further analysis!"}]

In [21]:
user_input = "for this customer id  10067724, show me the total transaction amount for each transaction type"
ai_chatbot(user_input, chat_history)


Function Arguments:
 {'user_requests': 'Show the total transaction amount for each transaction type for customer ID 10067724.', 'identifier': '10067724'}
No .env file found. Will use provided parameters or defaults.
Available SQL Server drivers: ODBC Driver 17 for SQL Server, SQL Server, SQL Server Native Client RDA 11.0
Attempting connection: windows auth using ODBC Driver 17 for SQL Server
Server: localhost\SQLEXPRESS, Database: master
✓ Successfully connected to SQL Server
  Server: localhost\SQLEXPRESS
  Database: master
  Authentication: windows
  Driver: ODBC Driver 17 for SQL Server
Running Query #1 ______________________
Executing query: SELECT 
    transaction_type,
    SUM(amount) AS total_amount
FROM [master].[dbo].[transaction_history] WITH (NOLOCK)
WHERE customer_id = 10067724
  AND status = 'Completed'
GROUP BY transaction_type
ORDER BY transaction_type;...


Query executed successfully - returned 1 rows with 2 columns


[{'role': 'user',
  'content': 'which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?'},
 {'role': 'assistant',
  'content': "Here's a summary of your requested database insights:\n\n1. **Quarter with the Highest and Lowest Amounts:**\n   - **Highest Amount:** Q4 2024, with a total of 1,608,701.77.\n   - **Lowest Amount:** Q2 2023, with a total of 91,590.26.\n\n2. **Customers with the Highest and Lowest Account Balances:**\n   - **Highest Balance:** Charles Cline (Account: 5423480814), with a balance of 49,859.96.\n   - **Lowest Balance:** Christy Adams (Account: 2310140863), with a balance of 158.62.\n\n3. **Channel with the Highest Transaction Volume:**\n   - **Check** channel, with 205 transactions.\n\nLet me know if you need more details or further analysis!"},
 {'role': 'user', 'content': 'what was my previous question? '},
 {'role': 'assistant',
  'content': 'Your pr

# Session id

In [ ]:


# Global session store (in-memory for simplicity)
session_store = {}

def ai_chatbot(user_input, session_id):
    # Retrieve or initialize session state
    session = session_store.get(session_id, {"chat_history": [], "sql_history": []})
    chat_history = session["chat_history"]

    if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_input:
        chat_history.append({"role": "user", "content": user_input})

    start_chat_run(user_input)

    try:
        response = routing_agent(user_input, chat_history)
        message = response.choices[0].message
        log_router_response(message)

        if hasattr(message, 'tool_calls') and message.tool_calls:
            tool_call = message.tool_calls[0]
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            sql_results = []

            if function_name == "agent_sql_analysis":
                sql_code = agent_sql_analysis(function_args["user_requests"])
                log_sql_code(sql_code)

                sql_results = execute_sql_with_pyodbc(sql_code, auth_type='windows')
                log_sql_results(sql_results)

                # Save SQL history
                session["sql_history"].append({
                    "request": function_args["user_requests"],
                    "sql": sql_code,
                    "results": sql_results
                })

            else:
                assistant_msg = "I don't know how to handle that request."
                chat_history.append({"role": "assistant", "content": assistant_msg})
                session_store[session_id] = session
                return chat_history

            # Process SQL results
            processed_sql_data = []
            for res in sql_results:
                if isinstance(res, dict) and res.get("status") == "success":
                    try:
                        json_data = json.loads(res["result"])
                        processed_sql_data.append({
                            "query_info": f"Query returned {res['row_count']} rows with columns: {', '.join(res['columns'])}",
                            "data": json_data
                        })
                    except json.JSONDecodeError:
                        processed_sql_data.append({
                            "query_info": f"Query returned {res['row_count']} rows",
                            "data": res["result"]
                        })
                else:
                    processed_sql_data.append({
                        "query_info": f"Query failed with status: {res.get('status', 'unknown')}",
                        "data": res.get("result", "No data available")
                    })

            if sql_results:
                polish_prompt = (
                    f"""Summarize the following database query results in a clear, friendly way based on the user requests:
                    ** User Requests:** {function_args.get("user_requests")} \n
                    ** Results: {processed_sql_data}"""
                )

                polish_response = agent_final_response(polish_prompt, [{"role": "user", "content": polish_prompt}])

                if polish_response and polish_response.choices and polish_response.choices[0].message:
                    final_message = polish_response.choices[0].message.content
                    log_final_response(final_message)
                    chat_history.append({"role": "assistant", "content": final_message})
                else:
                    fallback_message = "I've analyzed your request but encountered an issue formatting the results."
                    chat_history.append({"role": "assistant", "content": fallback_message})
                    log_final_response(fallback_message)
            else:
                assistant_msg = "I couldn't process your request. Please try again."
                chat_history.append({"role": "assistant", "content": assistant_msg})
                return chat_history

        else:
            content = message.content or "I don't have a response for that."
            log_final_response(content)
            chat_history.append({"role": "assistant", "content": content})

    finally:
        end_chat_run()

    # Save updated session
    session["chat_history"] = chat_history
    session_store[session_id] = session

    return chat_history


In [ ]:
chat_history = []
session_id = "user_001"

In [ ]:
user_input = "which quarter has the highest amount and the lowest amount? Which customer has the lowest and highest account balance? Which channel has the highest transaction volume?"
ai_chatbot(user_input, session_id)

# Access

In [ ]:
# AZURE_OPENAI_ENDPOINT = 'https://aoamanagement.azure-api.net/deployments/gpt-35-turbo/chat/completions?api-version=2023-08-01-preview'
# AZURE_OPENAI_VERSION = '2023-08-01-preview'
# AZURE_OPENAI_KEY = '627feee187a442b8b3dc4cd3d5803fb5'
# AZURE_OPENAI_DEPLOYMENT_NAME = 'gpt-35-turbo'

# AZURE_OPENAI_ENDPOINT = "https://aoamanagement.azure-api.net/za/deployments/gpt-4o-mini-ZA/chat/completions?api-version=2024-02-15-preview"
# AZURE_OPENAI_VERSION = '2024-02-15-preview'
# AZURE_OPENAI_KEY = '59d321feacac4b77895c138675a239b4'
# AZURE_OPENAI_DEPLOYMENT_NAME = 'gpt-4o-mini'

# Personal Account
# AZURE_OPENAI_ENDPOINT = 'https://rag-system-compliance-resource.cognitiveservices.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2025-01-01-preview'
# AZURE_OPENAI_VERSION = '2024-12-01-preview'
# AZURE_OPENAI_KEY = 'DFsvH8c1lrvCk6D2lTyBAM3OFXogRDdeKC8J3AxAX2FXIm3Mh9xkJQQJ99BBACHYHv6XJ3w3AAAAACOGCwmz'
# AZURE_OPENAI_DEPLOYMENT_NAME = 'o3-mini'

AZURE_OPENAI_ENDPOINT = https://20892-m7iyjxep-eastus2.cognitiveservices.azure.com/
AZURE_OPENAI_VERSION = 2025-01-01-preview
AZURE_OPENAI_KEY = DFsvH8c1lrvCk6D2lTyBAM3OFXogRDdeKC8J3AxAX2FXIm3Mh9xkJQQJ99BBACHYHv6XJ3w3AAAAACOGCwmz
AZURE_OPENAI_DEPLOYMENT_NAME = 'gpt-4.1'

MLFLOW_TRACKING_URI=http://127.0.0.1:8080